In [ ]:
try:
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.runnables import RunnableLambda, RunnableWithMessageHistory
    from langchain_core.chat_history import InMemoryChatMessageHistory
    from langchain_core.prompt_values import ChatPromptValue

    from langchain_google_genai import ChatGoogleGenerativeAI
    from langchain_groq import ChatGroq
    from dotenv import load_dotenv
    import requests
    import os
    from langchain_core.output_parsers import StrOutputParser
except:
    !pip install langchain langchain-google-genai langchain-groq
    from langchain_core.output_parsers import StrOutputParser
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.runnables import RunnableLambda, RunnableWithMessageHistory
    from langchain_core.chat_history import InMemoryChatMessageHistory
    from langchain_core.prompt_values import ChatPromptValue

    from langchain_google_genai import ChatGoogleGenerativeAI
    from langchain_groq import ChatGroq
    from dotenv import load_dotenv
    import requests
    import os

## LLMs

In [5]:
load_dotenv()

def call_apifreellm(prompt_value: ChatPromptValue) -> str:
    message = prompt_value.to_string()

    response = requests.post(
        "https://apifreellm.com/api/v1/chat",
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {os.getenv('APIFREELLM_API_KEY')}"
        },
        json={"message": message},
        timeout=30
    )

    response.raise_for_status()
    data = response.json()

    return data.get("response") or data.get("message")

apifreellm = RunnableLambda(call_apifreellm)

groq = ChatGroq(
    model="llama-3.3-70b-versatile"
)

gemma_3_1b = ChatGoogleGenerativeAI(
    model="gemma-3-1b-it",
    temperature=0.7,
    convert_system_message_to_human=True
)


### Chains

In [9]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who gives short and clear answers."),
    ("placeholder", "{history}"),
    ("human", "{question}")
])

store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

def build_memory_chain(llm):
    base_chain = prompt | llm | StrOutputParser()

    return RunnableWithMessageHistory(
        base_chain,
        get_chat_history,
        input_messages_key="question",
        history_messages_key="history",
    )

afl_chain = build_memory_chain(apifreellm)
groq_chain = build_memory_chain(groq)
gemma_chain = build_memory_chain(gemma_3_1b)

### Short Chat

In [11]:
config = {"configurable": {"session_id": "user_1"}}

response = gemma_chain.invoke({"question": "What is LangChain?"}, config=config)
print(response)

response = gemma_chain.invoke({"question": "Who created it?"}, config=config)
print(response)

LangChain is a tool that makes it easier to build applications using large language models. 

Think of it as a kit for creating intelligent applications! 
LangChain was created by Beijing Academy of Artificial Intelligence (BAAI).


In [12]:
response = groq_chain.invoke({"question": "Explain it more simply."}, config=config)
print(response)

LangChain is a tool that helps build apps using AI. It was made by a group called BAAI.


In [13]:
response = groq_chain.invoke({"question": "Give an example use case."}, config=config)
print(response)

Example: You can use LangChain to build a chatbot that answers customer questions, using a large language model to understand and respond to user queries.


In [14]:
response = groq_chain.invoke({"question": "Make it more challenging"}, config=config)
print(response)

Example: A company uses LangChain to build an AI-powered research assistant that can summarize complex documents, answer technical questions, and even generate code snippets, all using a large language model integrated with their existing knowledge base and workflow tools.
